# 循环神经网络

作者：杨岱川

时间：2020年03月

github：https://github.com/DrDavidS/basic_Machine_Learning

开源协议：[MIT](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/LICENSE)

参考文献：

- 《深度学习》，作者：Ian Goodfellow 、Yoshua Bengio、Aaron Courville。
- [CS231n](http://cs231n.stanford.edu/)
- 《动手学深度学习》，作者：李沐、阿斯顿·张

## 本节目的

多层感知机和卷积神经网络能够有效处理空间信息，而循环神经网络是为了更好地处理时序信息而设计。循环神经网络引入状态变量来保存过去的信息，然后与当前的输入共同决定当前的输出。

本节我们会学习循环神经网络的原理和使用方法。

## 序列数据与语言模型

首先我们会举例说明什么事序列数据，然后简单讲述一下语言模型。

### 序列数据是什么

一段文字、一段声音，或者电影胶片中前后相接的画面顺序，都是序列数据。可以认为有一些时间顺序和规律存在其中。

拿文字来说明，有这么一句话：

> 今天的天气真好呀

这句话就能被看做是一段离散的时间序列数据。这句话共有 $T = 8$ 个字，按索引来说，每个字我们表示为 $w_1, w_2, \cdots, w_T$，自然有 

> $w_1 = 今， w_2 = 天， \cdots，w_8 = 呀$

这样的一个序列可以用概率来表示，如 $P(w_1,w_2,\cdots,w_T)$，称为序列的概率。

在深度学习领域，对于语言文字的处理，或者叫 **自然语言处理（NLP）** 是非常热门的方向之一。在这个领域处理的数据大多都是类似的序列数据。

### 语言模型

**语言模型（Language Model）** 是自然语言处理的重要技术。上文中序列的概率 $P(w_1,w_2,\cdots,w_T)$ 就是通过语言模型计算出来的。

语言模型的应用十分广泛，比如“谷歌翻译”、“百度翻译”或者是一些智能问答系统等产品中都会使用语言模型来建模。

比如在英文翻译中常见的笑话：

> “How old are you？”

如果字对字地翻译会成为：

> “怎么老是你？”

当然按句意翻译实际应该是：

> “你多大了？”

------------

语言模型会针对这样的翻译文本计算一个概率。如果算出 “你多大了” 这个翻译结果的概率 $P$ 最大，那么我们就将

> “How old are you？”

翻译为

> “你多大了？”。

### 语言模型的建立

语言模型会怎么计算概率呢？这里我们要给出一个假设：

> 在序列 $w_1, w_2, \cdots, w_T$ 中，每个字（每个词）是依次生成的。

就如我们看书和说话一般，说了上一个字才说下一个字。

所以我们有：

$$\large P(w_1, w_2, \cdots, w_T) = \prod_{t=1}^{T}P(w_t|w_1,\cdots,w_{t-1})$$

比如有这么一句话 $S$:

> $S = 天气真好$

则我们有：

> $w_1 = 天， w_2 = 气， w_3 = 真，w_4 = 好$

那么这句话的概率就是：

$$\large P(S)=P(w_1, w_2, w_3, w_4)=P(w_1)P(w_2|w_1)P(w_3|w_1,w_2)P(w_4|w_1,w_2,w_3)$$

需要说明的是，对于中文不一定是**按字分割**，传统做法更多是**按词分割**，比如 $w_1=天气$ ，$w_2=真$，$w_3=好$。

### 语言模型的计算

建立好语言模型以后，我们需要实际进行计算。

那么，$P(w_1=天)$ 到底等于多少呢？

假设我们有一个大型的文本语料库，比如百度百科或者别的什么数据集，我们可以从中计算出“天”字出现的次数占语料库总字数比例。

比如语料库总共有 10000 个字（实际通常多得多），天字有 33 个。那么 ：

$$\large P(w_1=天) = 0.0033$$

接下来是 $P(w_2|w_1)$：

> $P(w_2=气|w_1=天)$ ：第一个字是“天”的情况下，第二个字是“气”的概率。

所以我们要统计在语料库的 33 个天字后面，有多少跟着“气”字。

假定有 5 个“天气”吧，所以有：

$$\large P(w_2=气|w_1=天) = 0.1515$$

以此类推，我们可以知道 $P(w_3|w_1,w_2)$ 就是要计算 “天气” 后面跟着 “真” 字的概率。

最后根据上面的公式，我们就能得到 $P(w_1, w_2, w_3, w_4)$ 的概率了。

### n元语法

当序列长度非常长的时候，计算和储存多个字（词）的概率会非常非常复杂，所以人们发明了 **n元语法（n-gram grammar）** 来简化语言模型的计算。

> **马尔科夫假设（Markov Assumption）**：一个字（词）的出现只和前面 $n$ 个字（词）有关。

这也叫做 **$n$ 阶马尔科夫链**。

如果 $n=1$，那么有 $P(w_3|w_1,w_2)=P(w_3|w_2)$。

如果基于 $n-1$ 阶马尔科夫链，我们可以把语言模型改写为

$$\large(w_1,w_2,\cdots,w_T) \approx \prod_{t=1}^{T}P(w_t|w_{t-(n-1)},\cdots,w_{t-1})$$

特别说明，当 $n$ 分别为 1、2、3 的时候，我们将其称为一元语法（unigram）、二元语法（bigram）、三元语法（trigram）。

以二元语法为例，之前的例子可以化简为：

$$\large P(w_1, w_2, w_3, w_4)=P(w_1)P(w_2|w_1)P(w_3|w_2)P(w_4|w_3)$$

当然我们要明白：

>当 $n$ 较小的时候，通常并不准确；而当 $n$ 较大的时候，计算和存储又会相当复杂，参数数量呈指数级增长。

## 循环神经网络

为了解决 n元语法 的问题，我们会介绍**循环神经网络**。

循环神经网络并非刚性地几亿所有固定长度的序列，而是通过隐藏状态来存储之前时间步的信息。

### 不含隐藏状态的神经网络

### 含隐藏状态的循环神经网络